In [1]:
using Revise
using DrWatson
@quickactivate "ABoxWorld"
include(srcdir("ABoxWorld.jl"));

┌ Info: ABoxWorld project environment is loaded and active
└ @ Main s:\Sync\University\2023_MRP_2\MRP2_WorkDir\ABoxWorld\src\ABoxWorld_core.jl:7


In [2]:
using LinearAlgebra
using Symbolics

### Creating NS Boxes
Defining a specific NS Box can be achieved in different ways.  <br>
As an interesting example beyond the CHSH scenerio, we consider a singlet in the 2233 scenerio. (see Collins et al, 2002, 10.1103/PhysRevLett.88.040404) <p> 
**Collins-Gisin (table) representation**

In [3]:
d=3 #Number of outcomes for each party
test_scenario = (2,2,d,d) #Scenario of the NSBox: (M_A, M_B, m_A, m_B) : M: number of measurement settings, m: number of outcomes per measurement

test_marginals_A =  [1/3, 1/3,  1/3, 1/3]
test_marginals_B =  [1/3, 1/3,  1/3, 1/3]
test_joint_mat = [(2/27*(2+sqrt(3)))        1/27                (2/27*(2+sqrt(3)))   (2/27*(2+sqrt(3)))  ; 
                  (2/27*(2-sqrt(3)))    (2/27*(2+sqrt(3)))            1/27           (2/27*(2+sqrt(3)))  ; 
    	                1/27            (2/27*(2+sqrt(3)))      (2/27*(2+sqrt(3)))         1/27          ; 
                  (2/27*(2-sqrt(3)))        1/27                (2/27*(2-sqrt(3)))   (2/27*(2+sqrt(3)))  ]

test_nsbox = nsboxes.NSBox(scenario=test_scenario,
                    marginals_vec_A=test_marginals_A,
                    marginals_vec_B=test_marginals_B,
                   joints_mat=test_joint_mat,
                   )

#test_nsbox = NSBox(scenario=(2,2,3,3), 
#                    marginals_vec_A=[0.1, 0.2, 0.3, 0.4],
#                    marginals_vec_B=[0.1, 0.2, 0.3, 0.4],
#                    joints_mat=[0.05 0.05 0.05 0.05; 0.05 0.05 0.05 0.05; 0.05 0.05 0.05 0.05; 0.05 0.05 0.05 0.05])


Main.nsboxes.NSBox((2, 2, 3, 3), [0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333], [0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333], [0.27644820796806496 0.037037037037037035 0.27644820796806496 0.27644820796806496; 0.019848088328231317 0.27644820796806496 0.037037037037037035 0.27644820796806496; 0.037037037037037035 0.27644820796806496 0.27644820796806496 0.037037037037037035; 0.019848088328231317 0.037037037037037035 0.019848088328231317 0.27644820796806496])

**Full Joint Distributions P(a,b|x,y)**

In [4]:
d=3 #Number of outcomes for each party
test_scenerio = (2,2,d,d)

#The following is eq. (14) from 10.1103/PhysRevLett.88.040404
alpha, beta = (0, 1/2), (1/4, -1/4)
P_abxy_maxEntangled =  Array{Float64}(undef, d, d, 2, 2)
for a in 0:d-1
    for b in 0:d-1
        for x in 1:2
            for y in 1:2
                P_abxy_maxEntangled[a+1,b+1,x,y] = 1/(2*d^3*sin((pi*(a-b+alpha[x]+beta[y]))/d)^2)
            end
        end
    end
end

display(P_abxy_maxEntangled)

test_nsbox = nsboxes.NSBox(test_scenerio,
                    P_abxy_maxEntangled)


3×3×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 0.276448   0.037037   0.0198481
 0.0198481  0.276448   0.037037
 0.037037   0.0198481  0.276448

[:, :, 2, 1] =
 0.037037   0.276448   0.0198481
 0.0198481  0.037037   0.276448
 0.276448   0.0198481  0.037037

[:, :, 1, 2] =
 0.276448   0.0198481  0.037037
 0.037037   0.276448   0.0198481
 0.0198481  0.037037   0.276448

[:, :, 2, 2] =
 0.276448   0.037037   0.0198481
 0.0198481  0.276448   0.037037
 0.037037   0.0198481  0.276448

Main.nsboxes.NSBox((2, 2, 3, 3), [0.3333333333333334, 0.3333333333333334, 0.3333333333333334, 0.3333333333333334], [0.3333333333333334, 0.3333333333333334, 0.3333333333333334, 0.3333333333333334], [0.2764482079680651 0.03703703703703705 0.2764482079680651 0.01984808832823131; 0.01984808832823131 0.2764482079680651 0.03703703703703705 0.2764482079680651; 0.03703703703703705 0.2764482079680651 0.2764482079680651 0.03703703703703705; 0.01984808832823131 0.03703703703703705 0.01984808832823131 0.2764482079680651])

Even if you create an NSBox from a full joint distribution $P(a,b|x,y)$, it is automatically converted into the more efficient CG represention. So if you need to get the full distribution back, you need to reconstruct it:

In [5]:

reconstruction = nsboxes.reconstructFullJoint(test_nsbox)
display(reconstruction - P_abxy_maxEntangled)

3×3×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 0.0          0.0           6.93889e-18
 0.0          0.0          -1.38778e-17
 6.93889e-18  6.93889e-18  -2.77556e-16

[:, :, 2, 1] =
 0.0          0.0           6.93889e-18
 0.0          0.0           0.0
 6.66134e-16  6.93889e-18  -2.91434e-16

[:, :, 1, 2] =
 0.0           0.0           6.93889e-18
 0.0           0.0           6.93889e-18
 6.93889e-18  -1.38778e-17  -2.77556e-16

[:, :, 2, 2] =
 0.0          0.0           6.93889e-18
 0.0          0.0          -1.38778e-17
 6.93889e-18  6.93889e-18  -2.77556e-16

Conveniently, we can compute and represent (convex) mixtures of (different) NS boxes by using simple multiplication and addition operators. Make sure to use a normalized set of coefficients between 0.0 and 1.0.

In [6]:
ubox = 0.2*test_nsbox + 0.8*test_nsbox

Main.nsboxes.NSBox((2, 2, 3, 3), [0.3333333333333335, 0.3333333333333335, 0.3333333333333335, 0.3333333333333335], [0.3333333333333335, 0.3333333333333335, 0.3333333333333335, 0.3333333333333335], [0.27644820796806513 0.03703703703703705 0.27644820796806513 0.01984808832823131; 0.01984808832823131 0.27644820796806513 0.03703703703703705 0.27644820796806513; 0.03703703703703705 0.27644820796806513 0.27644820796806513 0.03703703703703705; 0.01984808832823131 0.03703703703703705 0.01984808832823131 0.27644820796806513])

To minimize look-up of commonly used NS Boxes, some specific NS boxes can be created from convenience functions. <br>
Consider, for example, the canonical PR-boxes for the CHSH or any other (2,2,d,d) scenario:

In [7]:
UniformRandomBox((2,2,3,3))
MaxEntangledBoxCHSH() #Only for CHSH scenario

CanonicalPRBox() #Canoncal PR box in CHSH scenario
CanonicalPRBox(d=6) #Canoncal PR box in any (2,2,d,d) scenario, e.g. (2,2,6,6)

#display(reconstructFullJoint(CanonicalPRBox(2)))

Main.nsboxes.NSBox((2, 2, 6, 6), [0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666], [0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666], [0.16666666666666666 0.0 … 0.0 0.0; 0.0 0.16666666666666666 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.16666666666666666 0.0])

#### Families of NS Boxes

Families of NS Boxes are parameterizations of NS Boxes _within a specific scenerio_.
<p>
Also families of boxes can be easily created. Also here, you might either parameterize the joint distributions P(a,b|x,y) or the CG representation thereof, whatever seems more convenient. <p>
First define a custom function (called the "generator") that takes only keyword arguments, those correspond to the parameters of the NSBox family. Also make sure that your function always returns a NSBox instance!

In [8]:
function generator_fun(;p1, p2) #Generates NSBox instances, given some named parameters p1 and p2 (= keyword arguments!)
    return p1*CanonicalPRBox() + p2*UniformRandomBox((2,2,3,3))
end


generator_fun (generic function with 1 method)

You could work low-level with those generator functions. However, the basic operations that can be performed on NSBox instances don't work for generator functions. Therefore, you may want to make working with parameterized NS Boxes more convenient by embedding the generator function into a so-called NSBoxFamily instance. 

In [9]:
@variables p1, p2
ns_box_fam = nsboxes.NSBoxFamily((2,2,2,2), (p1, p2), generator_fun) #Creates a family of NSBoxes with parameters p1 and p2


Main.nsboxes.NSBoxFamily((2, 2, 2, 2), (p1, p2), Main.nsboxes.var"#safe_generator#15"{Main.nsboxes.var"#safe_generator#13#16"{typeof(generator_fun)}}(Main.nsboxes.var"#safe_generator#13#16"{typeof(generator_fun)}(generator_fun)))

Consequently, the feature-set and notation for working with those objects are almost equivalent to NSBox instances. 
<br>
Convex mixtures, for example, also work for PRBoxFamilies:

In [10]:
# As before, you can now do things like additon on NSBox families!
nsbox_fam_1 = 
nsbox_fam_2 =
mixed_fam = 0.5*nsbox_fam_1 + 0.5*nsbox_fam_2

UndefVarError: UndefVarError: `nsbox_fam_1` not defined

Of course, also the most common types of NSBox families already have some convenient implementation, both as a generator function as well as a NSBoxFamily instance. So those don't need to be defined again. Note that families are always restricted to a specific scenerio, mostly the CHSH (2,2,2,2) scenerio!  

In [11]:
#If you need local deterministic boxes for arbitrary scenarios:
test_scenario = (2,2,3,3)
test_generator = getLocalDeterministicBoxGenerator(test_scenario) #Retrieve a generator for local deterministic boxes in the given scenario

test_generator(;θ_a = [0 0;1 1], θ_b = [1 2; 1 2]) #Create some specific Local Deterministic NSBox. e.g. With a Dictionary: x=0, y=0 -> a=0, b=0; x=0, y=1 -> a=0, b=1; x=1, y=0 -> a=1, b=0; x=1, y=1 -> a=1, b=1
@variables θ_a[1:2,1:2], θ_b[1:2,1:2]
nsboxes.NSBoxFamily(test_scenario, (θ_a, θ_b), test_generator) #Define a NSBoxFamily on your own
LocalDeterministicBoxFamily(test_scenario) #Retrieve the (exact same) NSBoxFamily from a convenience function


#If you only need local deterministic boxes for the CHSH scenario, prefer for efficiency:
LocalDeterministicBoxesCHSH # The (fixed) generator function with parameters α, γ, β, λ ∈ {0,1}
LocalDeterministicBoxesCHSH(α=0, γ=0, β=0, λ=0) #Create a specific Local Deterministic NSBox from the generator function
@variables α, γ, β, λ
nsboxes.NSBoxFamily((2,2,2,2), (α, γ, β, λ), LocalDeterministicBoxesCHSH) #Define a NSBoxFamily on your own
LocalDeterministicBoxesCHSH_family() #Retrieve the (exact same) NSBoxFamily from a convenience function


#All 4 PR boxes in the CHSH scenario are equally accessible as:
PRBoxesCHSH # The (fixed) generator function with parameters μ, ν, σ ∈ {0,1}
PRBoxesCHSH(μ=0, ν=0, σ=0) #Create a specific PR NSBox from the generator function
@variables μ, ν, σ
nsboxes.NSBoxFamily((2,2,2,2), (μ, ν, σ), PRBoxesCHSH) #Define a NSBoxFamily on your own
PRBoxesCHSH_family() #Retrieve the (exact same) NSBoxFamily from a convenience function

#Istropic boxes (= mix of perfect PR Box and Uniform Random Box) in CHSH:
IsotropicBoxesCHSH # The (fixed) generator function with parameters p ∈ [0,1]
IsotropicBoxesCHSH(ρ=0.5) #Create a specific Isotropic NSBox from the generator function
@variables ρ
nsboxes.NSBoxFamily((2,2,2,2), (ρ, ), IsotropicBoxesCHSH) #Define a NSBoxFamily on your own
IsotropicBoxesCHSH_family() #Retrieve the (exact same) NSBoxFamily from a convenience function

Main.nsboxes.NSBoxFamily((2, 2, 2, 2), (ρ,), Main.nsboxes.var"#safe_generator#15"{Main.nsboxes.var"#safe_generator#13#16"{typeof(IsotropicBoxesCHSH)}}(Main.nsboxes.var"#safe_generator#13#16"{typeof(IsotropicBoxesCHSH)}(IsotropicBoxesCHSH)))

### Mixing NS Boxes

If we have multiple NS Boxes given, we might want to combine them by taking their convex mixture. This can be achieved by multiplication and addition and it works for both specific NS boxes and families of NS boxes: <br>

In [12]:
box1 = LocalDeterministicBoxesCHSH(α=0, γ=0, β=0, λ=0)
box2 = UniformRandomBox((2,2,2,2))
p1, p2 = 0.2, 0.8
mixedbox = p1*box1 + p2*box2

Main.nsboxes.NSBox((2, 2, 2, 2), [0.6000000000000001, 0.6000000000000001], [0.6000000000000001, 0.6000000000000001], [0.4 0.4; 0.4 0.4])

Make sure that the coefficients $p_i$ initialize a convex mixture ($p_i\geq 0$ and $\sum_{i} p_i = 1$), otherwise the addition operator will throw an exception:

In [13]:
p, q = 0.2, 0.4
mixedbox = p*box1 + q*box2

#NOTE: The following error is thus intended as a demo!

AssertionError: AssertionError: abs((coeff_1 + coeff_2) - 1.0) < global_eps_tol

In [14]:
0.5*CanonicalPRBox() + 0.5*LocalDeterministicBoxesCHSH(α=0, γ=0, β=0, λ=0)

Main.nsboxes.NSBox((2, 2, 2, 2), [0.75, 0.75], [0.75, 0.75], [0.75 0.75; 0.75 0.5])

You can also mix more than two NSBoxes. Also you can mix NSBoxes and NSBoxFamilies together to whatever degree you like:

In [15]:
function getNoisyPRBoxGenerator(extremal_noise_generator; extremal_noise_params...)
    scenario = (2,2,2,2)
    @assert extremal_noise_generator ∈ [LocalDeterministicBoxesCHSH, PRBoxesCHSH]
    function noisy_PRBox_generator(;c1::Float64, c2::Float64)
        #@show typeof(c1), typeof(c2), typeof(c1 + c2)
        @assert 0.0 - 10*eps() <= c1 <= 1.0 + 10*eps() "Got c1=$c1, but c1 must be in [0,1]!"
        @assert 0.0 - 10*eps() <= c2 <= 1.0 + 10*eps() "Got c2=$c2, but c2 must be in [0,1]!"
        @assert c1 + c2 <= 1.0 + 100*eps() "Got c1+c2=$(c1+c2), but c1 + c2 should not exceed 1.0!"
        return c1*CanonicalPRBox() + c2*extremal_noise_generator(;extremal_noise_params...) + (1.0-c1-c2)*UniformRandomBox(scenario)
        #return c2*extremal_noise_generator(;extremal_noise_params...) + (1-c1-c2)*UniformRandomBox(scenario)
    end
    return noisy_PRBox_generator
end

#LocalDeterministicBoxesCHSH (0000) and PRBoxesCHSH (010, 111)

noisy_PRBox_generator = getNoisyPRBoxGenerator(LocalDeterministicBoxesCHSH; 
                                                α=0, γ=0, β=0, λ=0)
noisy_PRBox_generator(c1=0.3, c2=0.1)

Main.nsboxes.NSBox((2, 2, 2, 2), [0.55, 0.55], [0.55, 0.55], [0.4 0.4; 0.4 0.25])

In [16]:
c1, c2 = 0.3, 0.1

nsboxes.NSBox((2,2,2,2), Float64.(c1*nsboxes.reconstructFullJoint(CanonicalPRBox()) + c2*nsboxes.reconstructFullJoint(LocalDeterministicBoxesCHSH(α=0, γ=0, β=0, λ=0)) + (1.0-c1-c2)*nsboxes.reconstructFullJoint(UniformRandomBox((2,2,2,2)))))

Main.nsboxes.NSBox((2, 2, 2, 2), [0.55, 0.55], [0.55, 0.55], [0.4 0.4; 0.4 0.25])

### Checking conditions / (in)equalities on correlations of NSBoxes

Let's check whether the maximally entangled state indeed satures the Tsirelson bound on the CHSH inequality:

In [17]:
res1 = conditions.evaluate(CHSH_Inequality(), MaxEntangledBoxCHSH())
res2 = conditions.evaluate(CHSH_Inequality(), CanonicalPRBox())
res3 = conditions.evaluate(CHSH_Inequality(), LocalDeterministicBoxesCHSH(α=0, γ=0, β=1, λ=1))
noisy_gen = getNoisyPRBoxGenerator(PRBoxesCHSH; μ=0, ν=1, σ=0)
#res4 = evaluate(CHSH_Inequality(), noisy_gen(c1=2.40/4, c2=1.5/4))
res4 = conditions.evaluate(Original_IC_Bound(), noisy_gen(c1=2.40/4, c2=1.5/4))
@show res1, res2, res3, res4

(res1, res2, res3, res4) = (2.82842712474619, 4.0, 2.0, 4.005000000000001)


(2.82842712474619, 4.0, 2.0, 4.005000000000001)

In [18]:
2.5/4 + 1.5/4

1.0

In [19]:
2*sqrt(2)

2.8284271247461903

In [20]:
res1 = conditions.evaluate(NPA_TLM_Criterion(), MaxEntangledBoxCHSH())
res2 = conditions.evaluate(NPA_TLM_Criterion(), CanonicalPRBox())
res3 = conditions.evaluate(NPA_TLM_Criterion(), LocalDeterministicBoxesCHSH(α=0, γ=0, β=0, λ=0))
res4 = conditions.evaluate(NPA_TLM_Criterion(), UniformRandomBox((2,2,2,2)))
@show res1, res2, res3, res4

(res1, res2, res3, res4) = (3.1415926535897927, 6.283185307179586, NaN, 0.0)


(3.1415926535897927, 6.283185307179586, NaN, 0.0)

In [21]:
conditions.check(CHSH_Inequality(), MaxEntangledBoxCHSH(), :Q)
#conditions.check(CHSH_Inequality(), CanonicalPRBox(), :Q)
#conditions.check(CHSH_Inequality(), LocalDeterministicBoxesCHSH(α=0, γ=0, β=1, λ=1), :Q)

true

In [23]:
Uffink_Bipartite_2222_Inequality().setBounds

Dict{Symbol, Int64} with 2 entries:
  :Q  => 4
  :NS => 8

Some pre-defined conditions on correlations:

In [24]:
CHSH_Inequality() #The (canonical) CHSH inequality; Variants from term signs s1, s2, s3, s4 ∈ {+1, -1}
Uffink_Bipartite_2222_Inequality()
#Almost_Quantum_Condition() #Only via SDP
#NPA_Q1_Bipartite_Condition() #Only via SDP


Main.conditions.GeneralizedCorrelationCondition((2, 2, 2, 2), var"#S_Uffink_Bipartite_2222#20"(), <=, missing, Dict(:Q => 4, :NS => 8))

#### Experiment 1 (Equiv. to original IC paper)

In [25]:

function exp1(noisy_PRBox_generatorm, first_chsh, second_chsh)

    boundary_params = Dict()
    CHSH_CHSH2_boundary_vals = Dict()

    eval_conditions = Dict("NPA_TLM" => NPA_TLM_Criterion(), "IC Orig." => Original_IC_Bound())

    for c_condition_key in keys(eval_conditions)
        boundary_params[c_condition_key] = []
        CHSH_CHSH2_boundary_vals[c_condition_key] = []

        c2_range::Vector{Union{Float64, Missing}} = collect(0.0:0.01:1.0)
        c1_stepsize = 1/3000
        c1_boundary::Vector{Union{Float64, Missing}} = []
        
        for c2 in c2_range
            
            boundary_c1_val = 0.0  # Find largest c1 such that the box is still in Q
            made_steps = 1 #Keep track of floating point precision on recurrent subtraction

            #First check whether we are outside Q already
            if !conditions.check(eval_conditions[c_condition_key], noisy_PRBox_generator(c1=boundary_c1_val, c2=c2), :Q)
                boundary_c1_val = missing
            #Next, check whether we are inside, but nevertheless already very very close to Q's boundary (essentially on it), otherwise start search for the boundary
            elseif abs(conditions.evaluate(eval_conditions[c_condition_key], noisy_PRBox_generator(c1=boundary_c1_val, c2=c2)) - eval_conditions[c_condition_key].setBounds[:Q]) > 10e3*eps() 
                while conditions.check(eval_conditions[c_condition_key], noisy_PRBox_generator(c1=boundary_c1_val, c2=c2), :Q)

                    boundary_c1_val += c1_stepsize
                    made_steps += 1
                    if boundary_c1_val > 1.0 - c2 + 10*eps() #Can't go beyond 1.0 - c2 = no valid convex mixture
                        #println(conditions.evaluate(CHSH_Inequality(s1=1, s2=-1, s3=1, s4=1), noisy_PRBox_generator(c1=boundary_c1_val, c2=c2)), " ",evaluate(CHSH_Inequality(), noisy_PRBox_generator(c1=boundary_c1_val, c2=c2)))
                        
                        boundary_c1_val -= c1_stepsize #Go back one step
                        
                        #Check whether we, nevertheless, are at the boundary
                        """Uncomment when want to label positivity boundaries as missing
                        if abs(evaluate(eval_conditions[c_condition_key], noisy_PRBox_generator(c1=boundary_c1_val, c2=c2)) - eval_conditions[c_condition_key].setBounds[:Q]) > 10e3*eps() 
                            #@show "Here", boundary_c1_val, c2
                            #@show evaluate(eval_conditions[c_condition_key], noisy_PRBox_generator(c1=boundary_c1_val, c2=c2)), eval_conditions[c_condition_key].setBounds[:Q]
                            boundary_c1_val = missing
                        end
                        """

                        break
                    end
                end
            end
            push!(boundary_params[c_condition_key], (boundary_c1_val, c2))
            
            if ismissing(boundary_c1_val)
                push!(CHSH_CHSH2_boundary_vals[c_condition_key], (missing, missing))
            else
                CHSH_val = conditions.evaluate(first_chsh, noisy_PRBox_generator(c1=boundary_c1_val, c2=c2))
                CHSH2_val = conditions.evaluate(second_chsh, noisy_PRBox_generator(c1=boundary_c1_val, c2=c2))
                push!(CHSH_CHSH2_boundary_vals[c_condition_key], (CHSH_val, CHSH2_val))
            end
        end

    end
    return eval_conditions, boundary_params, CHSH_CHSH2_boundary_vals
end




exp1 (generic function with 1 method)

In [38]:
# Plot CHSH2 vs. CHSH values for the boundary
using GLMakie

noisy_PRBox_generator = getNoisyPRBoxGenerator(PRBoxesCHSH; 
                                                μ=0, ν=1, σ=0)

eval_conditions,boundary_params, CHSH_CHSH2_boundary_vals = exp1(noisy_PRBox_generator, 
                                                CHSH_Inequality(),
                                                CHSH_Inequality(s1=1, s2=-1, s3=1, s4=1),
                                                )
                                                
cmap = [:deepskyblue3, :orangered, :gold, "#e82051"]

with_theme(palette = (; patchcolor = cgrad(cmap, alpha=0.45))) do
    fig1 = lines(collect(0.0:0.01:2.0), 4.0 .- collect(0.0:0.01:2.0); label = "N.S.", color=:black,
    axis = (; xlabel = "CHSH2", ylabel = "CHSH", title ="Title"),
    figure = (; size = (800,600), fontsize = 22))

    for condition_key in keys(eval_conditions)
        tlm_chsh, tlm_chsh2 = map(x -> x[1], CHSH_CHSH2_boundary_vals[condition_key]), map(x -> x[2], CHSH_CHSH2_boundary_vals[condition_key])
        lines!(tlm_chsh2, tlm_chsh; label = condition_key) #, color = :green
    end
    
    limits!(-0.001, 2.001, 2.001, 4.001)
    axislegend(; position = :ct, framecolor = :orange)

    #save("./assets/CHSH_vs_CHSH2_PR010.png", fig1)
    save(plotsdir("NSSliceBounds", "CHSH_vs_CHSH2_PR010.png"), fig1)
    fig1
end




In [39]:

noisy_PRBox_generator = getNoisyPRBoxGenerator(PRBoxesCHSH; 
                                                μ=1, ν=1, σ=0)


eval_conditions,boundary_params, CHSH_CHSH4_boundary_vals = exp1(noisy_PRBox_generator, 
                                                CHSH_Inequality(),
                                                CHSH_Inequality(s1=-1, s2=1, s3=1, s4=1),
                                                )

cmap = [:deepskyblue3, :orangered, :gold, "#e82051"]

with_theme(palette = (; patchcolor = cgrad(cmap, alpha=0.45))) do
    fig2 = lines(collect(0.0:0.01:2.0), 4.0 .- collect(0.0:0.01:2.0); label = "N.S.", color=:black,
    axis = (; xlabel = "CHSH4", ylabel = "CHSH", title ="Title"),
    figure = (; size = (800,600), fontsize = 22))

    for condition_key in keys(eval_conditions)
        tlm_chsh, tlm_chsh4 = map(x -> x[1], CHSH_CHSH4_boundary_vals[condition_key]), map(x -> x[2], CHSH_CHSH4_boundary_vals[condition_key])
        lines!(tlm_chsh4, tlm_chsh; label = condition_key) #, color = :green
    end
    
    limits!(-0.001, 2.001, 2.001, 4.001)
    axislegend(; position = :ct, framecolor = :orange)

    save(plotsdir("NSSliceBounds", "CHSH_vs_CHSH4_PR110.png"), fig2)
    fig2
end


In [40]:
noisy_PRBox_generator = getNoisyPRBoxGenerator(LocalDeterministicBoxesCHSH; 
                                                α=0, γ=0, β=0, λ=0)

eval_conditions,boundary_params, CHSH_CHSH2_boundary_vals = exp1(noisy_PRBox_generator,
                                                CHSH_Inequality(),
                                                CHSH_Inequality(s1=1, s2=-1, s3=1, s4=1),
                                                )

cmap = [:deepskyblue3, :orangered, :gold, "#e82051"]


with_theme(palette = (; patchcolor = cgrad(cmap, alpha=0.45))) do
    fig3 = lines(collect(0.0:0.01:2.0), 4.0 .- collect(0.0:0.01:2.0); label = "N.S.", color=:black,
    axis = (; xlabel = "CHSH2", ylabel = "CHSH", title ="Title"),
    figure = (; size = (800,600), fontsize = 22))

    for condition_key in keys(eval_conditions)
        tlm_chsh, tlm_chsh2 = map(x -> x[1], CHSH_CHSH2_boundary_vals[condition_key]), map(x -> x[2], CHSH_CHSH2_boundary_vals[condition_key])
        lines!(tlm_chsh2, tlm_chsh; label = condition_key) #, color = :green
    end
    
    limits!(-0.001, 2.001, 2.001, 4.001)
    axislegend(; position = :ct, framecolor = :orange)

    save(plotsdir("NSSliceBounds", "CHSH_vs_CHSH2_LD0000.png"), fig3)
    fig3
end

#### Experiment 2 (Equiv. to recent IC paper)

In [41]:
function double_noisy_PRBox(;q1, q2)
    @assert 0.0 - 10*eps() <= q1 <= 1.0 + 10*eps() 
    @assert 0.0 - 10*eps() <= q2 <= 1.0 + 10*eps()
    @assert q1 + q2 <= 1.0 + 100*eps()
    @show q1, q2, 1.0-q1-q2
    return (1.0-q1-q2)*CanonicalPRBox() + q1*LocalDeterministicBoxesCHSH(α=1, γ=0, β=0, λ=0) + q2*LocalDeterministicBoxesCHSH(α=1, γ=0, β=1, λ=0)
    #return c2*extremal_noise_generator(;extremal_noise_params...) + (1-c1-c2)*UniformRandomBox(scenario)
end

conditions.evaluate(Original_IC_Bound(), double_noisy_PRBox(q1=0.3, q2=0.05))

4.929999999999999

In [42]:



function exp2(noisy_PRBox_generator)

    boundary_params = Dict()

    eval_conditions = Dict("Uffink" => Uffink_Bipartite_2222_Inequality() ,"Pawlowski et al., 2009" => Original_IC_Bound(), "Jain et al., 2023" => Correlated_Inputs_IC_Bound(ϵ=0.99))

    for c_condition_key in keys(eval_conditions)
        boundary_params[c_condition_key] = []

        q1_range::Vector{Union{Float64, Missing}} = collect(0.0:0.001:1.0)
        q2_stepsize = 1/4000
        q2_boundary::Vector{Union{Float64, Missing}} = []
        
        for q1 in q1_range
            
            boundary_q2_val = 0.0  # Find largest q2 such that the box is still in Q

            #First check whether we are inside Q already
            if conditions.check(eval_conditions[c_condition_key], noisy_PRBox_generator(q2=boundary_q2_val, q1=q1), :Q)
                boundary_q2_val = missing
            #Next, check whether we are outside, but nevertheless already very very close to Q's boundary (essentially on it), otherwise start search for the boundary
            elseif abs(conditions.evaluate(eval_conditions[c_condition_key], noisy_PRBox_generator(q2=boundary_q2_val, q1=q1)) - eval_conditions[c_condition_key].setBounds[:Q]) > 10e3*eps() 
                while !conditions.check(eval_conditions[c_condition_key], noisy_PRBox_generator(q2=boundary_q2_val, q1=q1), :Q)

                    boundary_q2_val += q2_stepsize
                    
                    if boundary_q2_val > 1.0 - q1 + 10*eps() #Can't go beyond 1.0 - q1 = no valid convex mixture
                        
                        boundary_q2_val -= q2_stepsize #Go back one step

                        break
                    end
                end
            end
            push!(boundary_params[c_condition_key], (q1,boundary_q2_val))
            
        end

    end
    return eval_conditions, boundary_params
end



exp2 (generic function with 1 method)

In [46]:
using GLMakie

eval_conditions, boundary_params = exp2(double_noisy_PRBox)
@show boundary_params
cmap = [:gold, :deepskyblue3, :orangered, "#e82051"]

with_theme(palette = (; patchcolor = cgrad(cmap, alpha=0.45))) do

    fig = Figure(; size = (800,600), fontsize = 22)
    ax = Axis(fig[1, 1]; title = " ", xgridstyle = :dash, ygridstyle = :dash, xlabel = "q1", ylabel = "q2")
        
    for condition_key in keys(eval_conditions)
        q1_vals, q2_vals = map(q -> q[1], filter(q -> !ismissing(q[2]), boundary_params[condition_key])), map(q -> q[2], filter(q -> !ismissing(q[2]), boundary_params[condition_key]))
        #lines!(q1_vals, q2_vals; label = condition_key) #, color = :green
        
        band!(q1_vals, q2_vals, 1 .- q1_vals; label = condition_key)
    end
    
    #translate!(lineplot, 0, 0, 2) # move line to foreground
    limits!(-0.001, 1.001, 0.001, 0.301)
    axislegend(; position = :ct, framecolor = :orange)
    save(plotsdir("NSSliceBounds", "q1q2_space.png"), fig)
    fig
end

In [44]:

function exp1star(noisy_PRBox_generatorm, first_chsh, second_chsh)

    boundary_params = Dict()
    CHSH_CHSH2_boundary_vals = Dict()

    eval_conditions = Dict("NPA_TLM" => NPA_TLM_Criterion(), "IC Orig." => Original_IC_Bound(), "Jain et al., 2023" => Correlated_Inputs_IC_Bound(ϵ=0.99))

    for c_condition_key in keys(eval_conditions)
        boundary_params[c_condition_key] = []
        CHSH_CHSH2_boundary_vals[c_condition_key] = []

        c2_range::Vector{Union{Float64, Missing}} = collect(0.0:0.01:1.0)
        c1_stepsize = 1/3000
        c1_boundary::Vector{Union{Float64, Missing}} = []
        
        for c2 in c2_range
            
            boundary_c1_val = 0.0  # Find largest c1 such that the box is still in Q
            made_steps = 1 #Keep track of floating point precision on recurrent subtraction

            #First check whether we are outside Q already
            if !conditions.check(eval_conditions[c_condition_key], noisy_PRBox_generator(c1=boundary_c1_val, c2=c2), :Q)
                boundary_c1_val = missing
            #Next, check whether we are inside, but nevertheless already very very close to Q's boundary (essentially on it), otherwise start search for the boundary
            elseif abs(conditions.evaluate(eval_conditions[c_condition_key], noisy_PRBox_generator(c1=boundary_c1_val, c2=c2)) - eval_conditions[c_condition_key].setBounds[:Q]) > 10e3*eps() 
                while conditions.check(eval_conditions[c_condition_key], noisy_PRBox_generator(c1=boundary_c1_val, c2=c2), :Q)

                    boundary_c1_val += c1_stepsize
                    made_steps += 1
                    if boundary_c1_val > 1.0 - c2 + 10*eps() #Can't go beyond 1.0 - c2 = no valid convex mixture
                        #println(conditions.evaluate(CHSH_Inequality(s1=1, s2=-1, s3=1, s4=1), noisy_PRBox_generator(c1=boundary_c1_val, c2=c2)), " ",evaluate(CHSH_Inequality(), noisy_PRBox_generator(c1=boundary_c1_val, c2=c2)))
                        
                        boundary_c1_val -= c1_stepsize #Go back one step
                        
                        #Check whether we, nevertheless, are at the boundary
                        """Uncomment when want to label positivity boundaries as missing
                        if abs(conditions.evaluate(eval_conditions[c_condition_key], noisy_PRBox_generator(c1=boundary_c1_val, c2=c2)) - eval_conditions[c_condition_key].setBounds[:Q]) > 10e3*eps() 
                            #@show "Here", boundary_c1_val, c2
                            #@show conditions.evaluate(eval_conditions[c_condition_key], noisy_PRBox_generator(c1=boundary_c1_val, c2=c2)), eval_conditions[c_condition_key].setBounds[:Q]
                            boundary_c1_val = missing
                        end
                        """

                        break
                    end
                end
            end
            push!(boundary_params[c_condition_key], (boundary_c1_val, c2))
            
            if ismissing(boundary_c1_val)
                push!(CHSH_CHSH2_boundary_vals[c_condition_key], (missing, missing))
            else
                CHSH_val = conditions.evaluate(first_chsh, noisy_PRBox_generator(c1=boundary_c1_val, c2=c2))
                CHSH2_val = conditions.evaluate(second_chsh, noisy_PRBox_generator(c1=boundary_c1_val, c2=c2))
                push!(CHSH_CHSH2_boundary_vals[c_condition_key], (CHSH_val, CHSH2_val))
            end
        end

    end
    return eval_conditions, boundary_params, CHSH_CHSH2_boundary_vals
end




exp1star (generic function with 1 method)

In [45]:
# Plot CHSH2 vs. CHSH values for the boundary
using GLMakie

#noisy_PRBox_generator = getNoisyPRBoxGenerator(PRBoxesCHSH; 
#                                                μ=1, ν=1, σ=0)
noisy_PRBox_generator = getNoisyPRBoxGenerator(LocalDeterministicBoxesCHSH; 
                                                α=0, γ=0, β=0, λ=0)

eval_conditions,boundary_params, CHSH_CHSH2_boundary_vals = exp1star(noisy_PRBox_generator, 
                                                CHSH_Inequality(),
                                                CHSH_Inequality(s1=1, s2=-1, s3=1, s4=1),
                                                )
                                                
cmap = [:deepskyblue3, :orangered, :gold, "#e82051"]

with_theme(palette = (; patchcolor = cgrad(cmap, alpha=0.45))) do
    fig1 = lines(collect(0.0:0.01:2.0), 4.0 .- collect(0.0:0.01:2.0); label = "N.S.", color=:black,
    axis = (; xlabel = "CHSH2", ylabel = "CHSH", title ="Title"),
    figure = (; size = (800,600), fontsize = 22))

    for condition_key in keys(eval_conditions)
        tlm_chsh, tlm_chsh2 = map(x -> x[1], CHSH_CHSH2_boundary_vals[condition_key]), map(x -> x[2], CHSH_CHSH2_boundary_vals[condition_key])
        lines!(tlm_chsh2, tlm_chsh; label = condition_key) #, color = :green
    end
    
    limits!(-0.001, 2.001, 2.001, 4.001)
    axislegend(; position = :ct, framecolor = :orange)

    #save("./assets/CHSH_vs_CHSH2_PR010.png", fig1)
    fig1
end


